In [1]:
!pip install git+https://github.com/noc-lab/clinical_concept_extraction.git

  Cloning https://github.com/noc-lab/clinical_concept_extraction.git to /tmp/pip-req-build-34cadb_8
  Running command git clone -q https://github.com/noc-lab/clinical_concept_extraction.git /tmp/pip-req-build-34cadb_8
  Created wheel for clinical-concept-extraction: filename=clinical_concept_extraction-0.1.1-cp36-none-any.whl size=15454 sha256=dd410242a5e85dc4e520d5f6bb3f9ae830fd0394da9bfa78504d96cb31f43451
  Stored in directory: /tmp/pip-ephem-wheel-cache-8q1iy8wm/wheels/24/b8/02/aa7b358d164cfec41954c017081a6229e017c08935ae133844
Successfully built clinical-concept-extraction


In [2]:
!pip install onnxruntime

     |████████████████████████████████| 3.4MB 2.7MB/s eta 0:00:01


In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from clinical_concept_extraction.elmo_vector import ELMO_MIMIC

In [4]:
with open('/opt/conda/lib/python3.6/site-packages/clinical_concept_extraction/pipeline.py', 'r') as file :
    filedata = file.read()

filedata = filedata.replace('tf.data.Dataset()', 'tf.data.Dataset')

with open('/opt/conda/lib/python3.6/site-packages/clinical_concept_extraction/pipeline.py', 'w') as file:
    file.write(filedata)
    
os.environ["CCE_ASSETS"] = "../input/cce-assets/cce_assets/cce_assets/"

In [6]:
data = pd.read_csv("../input/cadec-v1/CADEC.csv",encoding="latin1")
data = data.fillna(method = "ffill")

In [8]:
data.tail(100)

,Document ID,Sentence No,Word,Tag
122480,ZIPSOR.5,7516,",",O
122481,ZIPSOR.5,7516,and,O
122482,ZIPSOR.5,7516,for,O
122483,ZIPSOR.5,7516,which,O
122484,ZIPSOR.5,7516,I,O
...,...,...,...,...
122575,ZIPSOR.5,7519,no,O
122576,ZIPSOR.5,7519,recognizable,O
122577,ZIPSOR.5,7519,side,O
122578,ZIPSOR.5,7519,effects,O


In [9]:
def sent_get(data):
    n_sent = 1
    data = data
    empty = False
    agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(), s["Tag"].values.tolist())]
    grouped  = data.groupby("Sentence No").apply(agg_func)
    sentences = [s for s in grouped]
    
    return sentences

sentences = sent_get(data)

In [10]:
len(sentences)

7520

In [12]:
tags = list(set(data["Tag"].values))
n_tags = len(tags)
n_tags

11

In [13]:
tag2idx = {t: i for i, t in enumerate(tags)}

In [14]:
sentences[0]

[('I', 'O'),
 ('feel', 'O'),
 ('a', 'O'),
 ('bit', 'B-ADR'),
 ('drowsy', 'I-ADR'),
 ('&', 'O'),
 ('have', 'O'),
 ('a', 'O'),
 ('little', 'B-ADR'),
 ('blurred', 'I-ADR'),
 ('vision', 'I-ADR'),
 (',', 'O'),
 ('so', 'O'),
 ('far', 'O'),
 ('no', 'O'),
 ('gastric', 'B-ADR'),
 ('problems', 'I-ADR'),
 ('0', 'O')]

In [15]:
def ElmoEmbedding(ip,elmo_models):
    out = []

    for sentence in ip:
        embeddings = elmo_models.get_embeddings(sentence)
        embeddings = np.mean(embeddings,axis=2)
        out.append(embeddings)

    return np.array(out)

In [38]:
from keras.preprocessing.sequence import pad_sequences

X = [[w[0] for w in s] for s in sentences]
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [39]:
X

[['I',
  'feel',
  'a',
  'bit',
  'drowsy',
  '&',
  'have',
  'a',
  'little',
  'blurred',
  'vision',
  ',',
  'so',
  'far',
  'no',
  'gastric',
  'problems',
  '0'],
 ['I',
  "'",
  've',
  'been',
  'on',
  'Arthrotec',
  '50',
  'for',
  'over',
  '10',
  'years',
  'on',
  'and',
  'off',
  ',',
  'only',
  'taking',
  'it',
  'when',
  'I',
  'needed',
  'it',
  '0'],
 ['Due',
  'to',
  'my',
  'arthritis',
  'getting',
  'progressively',
  'worse',
  ',',
  'to',
  'the',
  'point',
  'where',
  'I',
  'am',
  'in',
  'tears',
  'with',
  'the',
  'agony',
  ',',
  'gp',
  "'",
  's',
  'started',
  'me',
  'on',
  '75',
  'twice',
  'a',
  'day',
  'and',
  'I',
  'have',
  'to',
  'take',
  'it',
  '0'],
 ['every',
  'day',
  'for',
  'the',
  'next',
  'month',
  'to',
  'see',
  'how',
  'I',
  'get',
  'on',
  ',',
  'here',
  'goes',
  '0'],
 ['So',
  'far',
  'its',
  'been',
  'very',
  'good',
  ',',
  'pains',
  'almost',
  'gone',
  ',',
  'but',
  'I',
  'feel',

In [40]:
max_len = 50
X = pad_sequences(maxlen=max_len, dtype=object, sequences=X, padding="post", value="ENDPAD")
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [41]:
i = 0
for a in X:
    if(len(a) > 50):
        print(len(a),i)
    i = i+1

In [21]:
elmo_models = ELMO_MIMIC()

In [42]:
X

array([['I', 'feel', 'a', ..., 'ENDPAD', 'ENDPAD', 'ENDPAD'],
       ['I', "'", 've', ..., 'ENDPAD', 'ENDPAD', 'ENDPAD'],
       ['Due', 'to', 'my', ..., 'ENDPAD', 'ENDPAD', 'ENDPAD'],
       ...,
       ['The', 'Zipsor', 'works', ..., 'ENDPAD', 'ENDPAD', 'ENDPAD'],
       ['As', 'with', 'any', ..., 'ENDPAD', 'ENDPAD', 'ENDPAD'],
       ['I', 'have', 'had', ..., 'ENDPAD', 'ENDPAD', 'ENDPAD']],
      dtype=object)

In [43]:
X = ElmoEmbedding(X,elmo_models)

In [44]:
X.shape

(7520, 50, 1024)

In [ ]:
elmo_models.close_session()

In [45]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [46]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)

In [47]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-mxws2euc
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-mxws2euc
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=c72024479bc9642ffa7adf4a2e74b27bb087b2eed0876c200a5e5976db734057
  Stored in directory: /tmp/pip-ephem-wheel-cache-xq4xvdjv/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [48]:
X_tr.shape

(6016, 50, 1024)

In [49]:
from keras.models import Model, Input, Sequential
from keras.layers import Lambda, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.layers.merge import add
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

In [ ]:
ElmoEmbedding("I took crocin")[0].shape

In [50]:
input_text = Input(shape=(max_len,1024))
x = Bidirectional(LSTM(units=512, return_sequences=True, recurrent_dropout=0.2, dropout=0.2))(input_text)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True, recurrent_dropout=0.2, dropout=0.2))(x)
x = add([x, x_rnn])  # residual connection to the first
biLSTMmodel = TimeDistributed(Dense(512, activation="relu"))(x)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(biLSTMmodel)  # output

In [51]:
model = Model(input_text, out)
model.compile(optimizer="rmsprop", loss=crf_loss, metrics=[crf_viterbi_accuracy])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 1024)     0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 50, 1024)     6295552     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 50, 1024)     6295552     bidirectional_1[0][0]            
__________________________________________________________________________________________________
add_1 (Add)                     (None, 50, 1024)     0           bidirectional_1[0][0]            
                                                                 bidirectional_2[0][0]      

In [52]:
history = model.fit(np.array(X_tr), np.array(y_tr), batch_size=32, epochs=5, validation_split=0.1, verbose=1)

Train on 5414 samples, validate on 602 samples
Epoch 1/5
5414/5414 [==============================] - 69s 13ms/step - loss: 0.1328 - crf_viterbi_accuracy: 0.9580 - val_loss: 0.0874 - val_crf_viterbi_accuracy: 0.9735
Epoch 2/5
5414/5414 [==============================] - 67s 12ms/step - loss: 0.0546 - crf_viterbi_accuracy: 0.9761 - val_loss: 0.0612 - val_crf_viterbi_accuracy: 0.9749
Epoch 3/5
5414/5414 [==============================] - 66s 12ms/step - loss: 0.0328 - crf_viterbi_accuracy: 0.9806 - val_loss: 0.0644 - val_crf_viterbi_accuracy: 0.9749
Epoch 4/5
5414/5414 [==============================] - 66s 12ms/step - loss: 0.0130 - crf_viterbi_accuracy: 0.9853 - val_loss: 0.0746 - val_crf_viterbi_accuracy: 0.9734
Epoch 5/5
5414/5414 [==============================] - 67s 12ms/step - loss: -7.2501e-04 - crf_viterbi_accuracy: 0.9900 - val_loss: 0.0565 - val_crf_viterbi_accuracy: 0.9759


In [53]:
!pip install seqeval
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=b8070625e6a5afe7894495bfba04f5c28f7be5c0fa875eff0b4c96d084171ffd
  Stored in directory: /tmp/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [54]:
test_pred = model.predict(X_te, verbose=1)
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
print(classification_report(test_labels, pred_labels))

1504/1504 [==============================] - 9s 6ms/step
F1-score: 68.4%
           precision    recall  f1-score   support

      ADR       0.65      0.71      0.68      1122
     Drug       0.95      0.90      0.92       335
  Finding       0.30      0.37      0.33        90
  Disease       0.67      0.10      0.17        62
  Symptom       0.53      0.14      0.22        58

micro avg       0.68      0.68      0.68      1667
macro avg       0.69      0.68      0.67      1667



In [66]:
i = 1
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_te[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], true, p[0]):
    print(tags[t],tags[pred])
#     print("{:15}: {:5} {}".format(tags[t], tags[pred]))

Word           ||True ||Pred
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
B-Drug B-Drug
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
